In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from google.colab import drive
drive.mount("/content/gdrive")
%cd gdrive/MyDrive/Ml/nbs/dl2/

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)


def get_data():
    import os
    import torchvision.datasets as datasets
    root = '../data'
    if not os.path.exists(root):
        os.mkdir(root)
    train_set = datasets.MNIST(root=root, train=True, download=True)
    test_set = datasets.MNIST(root=root, train=False, download=True)
    x_train, x_valid = train_set.data.split([50000, 10000])
    y_train, y_valid = train_set.targets.split([50000, 10000])
    return (x_train.view(50000, -1) / 256.0), y_train.float(), (x_valid.view(10000, -1))/ 256.0, y_valid.float()

x_train,y_train,x_valid,y_valid = get_data()

Mounted at /content/gdrive
/content/gdrive/MyDrive/Ml/nbs/dl2


In [ ]:
#export
from exp.nb_03 import *
import torchvision

## DataBunch/Learner

[Jump_to lesson 9 video](https://course19.fast.ai/videos/?lesson=9&t=4799)

In [ ]:
# x_train,y_train,x_valid,y_valid = get_data()

train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c
        
        #c is the number of classes
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [ ]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)
# get_dls is from torch.dataloader
# torch dataloader use dataset instead of ds which we studied before with making our own function

# def get_dls(train_ds, valid_ds, bs, **kwargs):
#     return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
#             DataLoader(valid_ds, batch_size=bs*2, **kwargs))

# c is maximum y value

In [ ]:
# def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
#     for epoch in range(epochs):
        
#         model.train()

#         for xb,yb in train_dl:
#             loss = loss_func(model(xb), yb)
#             loss.backward()
#             opt.step()
#             opt.zero_grad()

#         model.eval()

#         with torch.no_grad():
#             tot_loss,tot_acc = 0.,0.
#             for xb,yb in valid_dl:
#                 pred = model(xb)
#                 tot_loss += loss_func(pred, yb)
#                 tot_acc  += accuracy (pred,yb)
#         nv = len(valid_dl)
#         print(epoch, tot_loss/nv, tot_acc/nv)
#     return tot_loss/nv, tot_acc/nv

Factor out the connected pieces of info out of the fit() argument list

`fit(epochs, model, loss_func, opt, train_dl, valid_dl)`

    ` 1. pass them around to everything that needs them together `

    ` 2. create them easily together and look at the combination of them and make smart decision things together`

Let's replace it with something that looks like this:

`fit(1, learn)`

This will allow us to tweak what's happening inside the training loop in other places of the code because the `Learner` object will be mutable, so changing any of its attribute elsewhere will be seen in our training loop.

[Jump_to lesson 9 video](https://course19.fast.ai/videos/?lesson=9&t=5363)

In [ ]:
#export
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1] #size of input

    # model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c)) original
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,int(data.c)))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

# learner is storage device

In [ ]:
learn = Learner(*get_model(data), loss_func, data)
# we need to fix the error due to different type
# data.c is float so we need to change it to int

In [ ]:
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            # print(xb,yb)
            loss = learn.loss_func(learn.model(xb), yb.long()) # error arose 바꿔야함
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss,tot_acc = 0,0
            for xb,yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred, yb.long())
                tot_acc  += accuracy (pred,yb)
        nv = len(learn.data.valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv  

In [ ]:
loss,acc = fit(2, learn)

0 tensor(0.2565) tensor(0.9152)
1 tensor(0.1452) tensor(0.9578)


## Callback

In [ ]:
# why need callbacks
# callback : backward direction - automatic system call the function if condition satistfied.
# modify the source code directly is dangerous so we need to use the other way to make events if some condition is satisfied

In [ ]:
# 7 images

In [ ]:
class test():
    def test1(self,in2,in3):
        print(in2,in3)
        return 
class inherited_test(test):
    def test1(self,in2,in3):
        print(in2,in3)
        print('moremore')
        return

test().test1('1','2')
print('------')
inherited_test().test1('1','2')

1 2
------
1 2
moremore


In [ ]:
# original
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        # for xb,yb in learn.data.train_dl:  extract to all batch

            # loss = learn.loss_func(learn.model(xb), yb.long())   ====> extract to one batch 
            # loss.backward()
            # learn.opt.step()
            # learn.opt.zero_grad()
    # why extract and make them fucntion ? -> just make code simple and intuitive by refactoring
        learn.model.eval()
        with torch.no_grad():
        #     tot_loss,tot_acc = 0,0                  ====> also extract to one batch
        #     for xb,yb in learn.data.valid_dl:
        #         pred = learn.model(xb)
        #         tot_loss += learn.loss_func(pred, yb.long())
        #         tot_acc  += accuracy (pred,yb)
        # nv = len(learn.data.valid_dl)
        # print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv  

IndentationError: ignored



```python
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()
    
def fit():
    for epoch in range(epochs):
        for b in train_dl: one_batch(*b)
```

Add callbacks so we can remove complexity from loop, and make it flexible:

[Jump_to lesson 9 video](https://course19.fast.ai/videos/?lesson=9&t=5628)

In [ ]:

def one_batch(xb,yb):
    #we want to execute some process in ceratin condition here -> maybe stop in certain batch
    pred = model(xb)
    #we want to execute some process in ceratin condition here -> maybe change the type of preds
    loss = loss_func(pred, yb)
    #we want to execute some process in ceratin condition here
    loss.backward()
    #we want to execute some process in ceratin condition here 
    opt.step()
    #we want to execute some process in ceratin condition here -> gradient accumulation
    opt.zero_grad()



def all_batches(dl, cb):
    for xb,yb in dl:
        one_batch(xb, yb, cb)
        #we want to execute some process in ceratin condition here after the one batch


def fit(epochs, learn, cb):
    #we want to execute some process in ceratin condition here before going into training
    for epoch in range(epochs):
        # self.in_train = True
        #we want to execute some process in ceratin condition here in cetain epoch -> not only learn.model.train() also other event 
        all_batches(learn.data.train_dl, cb)
        
        
        #we want to execute some process in ceratin condition here in before validating -> not only learn.model.eval() also other event 
        with torch.no_grad(): 
            # self.in_train = false
            all_batches(learn.data.valid_dl, cb)
        #we want to execute some process in ceratin condition here in after finishing epoch 


In [ ]:
# skip it

## hooks 
# in the operating system or any applications, changing or interrupting calls/events/message between the software components means hooking
# and to do hooking, we use some codes called hook.

# delegate 
# delegation refers generally to one entity passing something to another entity(function or variable), and narrowly to various specific forms of relationships.

###

In [ ]:
# whole those event codes in fit method makes code messy.
# And sometimes those codes may attack or bother the fit processes. So we want handle those event efficiently
# At this point, we need something to handle those events independently called callbacks


# first set the frame where we want make event in certain condition. That is, partition the point of fit process.
# this is callback structure
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): 
        return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

# whole returns true bcz if not true they will interrupt the process
# and in the literally thinking, system ask to fit process begin_fit ? and say yes.

In [ ]:
# we can change the function with same input and output but different content of function
# which is what we want execute when fit process in certain condition
class TestCallback(Callback):

    def begin_fit(self, learn):
        super().begin_fit(learn)
        print(f'{"epoch":<20}{"batch":<20}{"trainloss":<40}{"train_error":<40}')
        return True


    def begin_epoch(self,epoch):
        super().begin_epoch(epoch)
        self.n_iters = 0
        print('{:<20}'.format(self.epoch+1),end='')
        return True

    def begin_batch(self, xb, yb):
        super().begin_batch(xb,yb)
        self.tot_loss, self.tot_acc = 0,0 
        return True

    def after_step(self):
        self.n_iters += 1
        pred = self.learn.model(self.xb)
        self.tot_loss = self.learn.loss_func(pred, self.yb.long())
        self.tot_acc = accuracy(pred,self.yb)
        nv = len(self.yb)
        if self.n_iters == 1:
            print('{:<20}{:<40}{:<40}'.format(self.n_iters, (self.tot_loss/nv).item(),(self.tot_acc/nv).item()))
        else:
            print('{:<20}{:<20}{:<40}{:<40}'.format('', self.n_iters, (self.tot_loss/nv).item(),(self.tot_acc/nv).item()))
        return True




In [ ]:
# we may have lots of callbacks not the only one callback so we need one class more handle the callbacks entirely.
# Thats why we need callbackhandler

class CallbackHandler():
    # get multiple callbacks which is cbs
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []
            
    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True # stick the learn with callbacks.
        learn.stop = False
        res = True # result
        for cb in self.cbs: 
            res = (res and cb.begin_fit(learn)) 

        return res

    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: 
            res = res and cb.begin_epoch(epoch)
        return res

    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: 
            res = res and cb.after_step() # true and something

        return res

    def do_stop(self):
        try:    
             return self.learn.stop
        finally: 
            self.learn.stop = False

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    




        


In [ ]:
# suppose we have only one callback and apply it into batch

def one_batch(xb, yb, cb):
    if not cb.begin_batch(xb,yb): return

    loss = cb.learn.loss_func(cb.learn.model(xb), yb.long())

    if not cb.after_loss(loss): return
    loss.backward()

    if cb.after_backward(): cb.learn.opt.step()

    if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
    for (xb,yb) in dl:
        one_batch(xb, yb, cb)
        if cb.do_stop(): return

def fit(epochs, learn, cb):
    if not cb.begin_fit(learn): # if begin_fit returns false, interrupt the fit process
        return # === return None

    for epoch in range(epochs):

        if not cb.begin_epoch(epoch): 
            continue
        all_batches(learn.data.train_dl, cb)
        
        if cb.begin_validate():
            with torch.no_grad():
                all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): 
            break
    cb.after_fit()

In [ ]:
fit(2, learn, cb=CallbackHandler([TestCallback()]))

This is roughly how fastai does it now (except that the handler can also change and return `xb`, `yb`, and `loss`). But let's see if we can make things simpler and more flexible, so that a single class has access to everything and can change anything at any time. The fact that we're passing `cb` to so many functions is a strong hint they should all be in the same class!

## Runner

[Jump_to lesson 9 video](https://course19.fast.ai/videos/?lesson=9&t=5811)

In [ ]:
# jeremy thinks he can improve his code including 
# 1. cb input in many function multiple times.
# 2. so many duplicate codes in callback
# 3. better process than just inherit all things(not forcing all of things to inherit) => if some of function is missed, error will arise

# one_batch(xb, yb, cb):  ====> self('event name')
# all_batches(dl, cb):       ===> get rid of necessity of keeping state cb.
# fit(epochs, learn, cb):


# duplicate codes

# res = True                                        ====> callback.__call__() : treat object as a function
# for cb in self.cbs: res = res and cb.after_epoch()
# return res

# callbacks becomes super simple than before
# class Callback():
#     _order=0 # choose the order of callbacks' running 
#     def set_runner(self, run): 
#         # print('callback',self,run)
#         self.run=run
#     def __getattr__(self, k): return getattr(self.run, k) # if python cannot find go into this dunder code
#     #@property
#     def name(self): # name change
#         name = re.sub(r'Callback$', '', self.__class__.__name__)
#         return camel2snake(name or 'callback')

In [ ]:
# He makes new class named runner which is improved version

# there is big differencec that former callback returns true in default but now runner's condition is to return false
#former
# if not cb.begin_batch(xb,yb): return
#latter
# if self('begin_fit'): return

# the reason is that in general function without return returns None which is false. So for preventing mistake or any omittion he changed


# 
# dunder call -> double underline function
# __call__, __init__, __getattr__

In [ ]:
#export
from typing import *


## it makes o into list type even if it is string, None, generator
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

# run = Runner(cbs=stats)
# run = Runner(cbs=[stats])
# run = Runner(cbs=None)
# run = Runner(cbs='metric name')
# run = Runner(cbs=generator)
# all cbs could be handled as list

In [ ]:
## runner contains 
# 1. one batch
# 2. all batch
# 3. callbackhandlers => __call__ #dunder method == __something__
# 4. fit
## two types
class Runner(): 
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs) # cbs is initialized callback
        for cbf in listify(cb_funcs):  # cb_funcs is not the initialized yet
            cb = cbf()
            setattr(self, cb.name, cb) # automatically sets the callbacks to runner, that is stick together
            # print(cb.name)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data

    def one_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        if self('begin_batch'): return
        self.pred = self.model(self.xb)
        if self('after_pred'): return
        self.loss = self.loss_func(self.pred, self.yb.long())
        if self('after_loss') or not self.in_train: return  #if not cb.after_loss(loss): return  => we don't need cb
        self.loss.backward()
        if self('after_backward'): return
        self.opt.step()
        if self('after_step'): return
        self.opt.zero_grad()

    def all_batches(self, dl):
        self.iters = len(dl)
        for xb,yb in dl:
            if self.stop: break
            self.one_batch(xb, yb)
            self('after_batch')
        self.stop=False


    def fit(self, epochs, learn):
        self.epochs,self.learn = epochs,learn #keep track epochs and initialize the learner

        try:
            for cb in self.cbs: 
                # print('fit',self)
                cb.set_runner(self) #assign runner to callback.run // it makes the callback link to runner(self). -> Callback can use the runner's variables due to __getattr__
            if self('begin_fit'): 
                return  #self('begin_fit') -> runner instance('begin_fit')
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad(): 
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                if self('after_epoch'): break
            
        finally:
            self('after_fit')
            self.learn = None

    # duplicate callback codes into dunder function
    # by this, we need not to input the cb multiple times.
    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None) # if callback's function name doesn't exist in callback, then return None
            # if cb_name is not in cb set the cb_name is None
            if f and f(): return True
        return False

In [ ]:
#export
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

# TrainEvalCallback
# -> train_eval

In [ ]:
cbname = 'TrainEvalCallback'
camel2snake(cbname)

'train_eval_callback'

In [ ]:
cbname = ''
cbname or 'callback'

'callback'

In [ ]:
class Callback():
    _order=0 # choose the order of callbacks' running 
    def set_runner(self, run): 
        # print('callback',self,run)
        self.run=run
    def __getattr__(self, k): return getattr(self.run, k) # if python cannot find in callback, then go into this dunder code
                                                        # callback.run.something  ==> kind of delegate concept
    @property
    def name(self): # callback class name change into some format
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

This first callback is reponsible to switch the model back and forth in training or validation mode, as well as maintaining a count of the iterations, or the percentage of iterations ellapsed in the epoch.

In [ ]:
# most basic callback 
# they set the basic training loop with tracing the iteration count and epoch count
class TrainEvalCallback(Callback): 
    # _order=0 which is inherited from callback
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False
TrainEvalCallback().name

'train_eval'

We'll also re-create our TestCallback

In [ ]:
class TestCallback(Callback):  
    _order = 1 # bcz TestCallback should execute after TrainEvalCallback 
    def after_step(self):
        if self.n_iter>=10: return True # we can use this instead of self.run.n_iter

Third callback: how to compute metrics.

In [ ]:

class AvgStats():
    def __init__(self, metrics, in_train):
         self.metrics,self.in_train = listify(metrics),in_train
    
    def reset(self):
        self.tot_loss,self.count = 0.,0
        # print('metrics',len(self.metrics))
        self.tot_mets = [0.] * len(self.metrics) # [0.] duplicate by len(self.metrics)
        # print('tot_mets',self.tot_mets)

    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets # loss and accuracy
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats] # get the mean of metrics
    
    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0] # 64,784
        self.tot_loss += run.loss * bn # loss total(summation of loss) / batch size = run.loss
        self.count += bn
        for i,m in enumerate(self.metrics):
            #
            self.tot_mets[i] += m(run.pred, run.yb) * bn

# 1 batch 1 get loss one batch
# accumulate
# total loss      count.  tot_loss
# 1batch loss. // 64.     1batch
# 1batch loss. // 128.    1batch + 1batch
# 1batch loss. // 192.     1batch + 1batch + 1batch loss


class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
        
    # initialize the metric and loss parameters as soon as epoch starts
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
    
    # if training loop, use train_stats. If validating loop, use valid_stats
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
        
    # after each epoch, print the loss and accuracy
    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)

In [ ]:
learn = Learner(*get_model(data), loss_func, data)

In [ ]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [ ]:
run.fit(4, learn)

train: [0.138389296875, tensor(0.9582)]
valid: [0.496626416015625, tensor(0.8675)]
train: [0.105518388671875, tensor(0.9682)]
valid: [0.10035296630859375, tensor(0.9693)]
train: [0.0863809765625, tensor(0.9727)]
valid: [0.113865478515625, tensor(0.9658)]
train: [0.0729070751953125, tensor(0.9771)]
valid: [0.2067080322265625, tensor(0.9445)]


In [ ]:
loss,acc = stats.valid_stats.avg_stats
assert acc>0.9
loss,acc

(0.2067080322265625, tensor(0.9445))

In [ ]:
# with this process, we have something to improve 
# it is awkward to deal with a callback instance and metric independenlty. so we can put metrics into the runner class then refer to the runner.something not stats.something
# so we can merge whole processes into one instance
# that is why there is cb_func parameter for runner.

In [ ]:
# 
from functools import partial
## https://hamait.tistory.com/823 partial example

# what is partial?

# def power(base, exponent):
#     return base ** exponent

# def square(base):                    => partial(power,exponent=2)
#     return power(base, 2)

# def cube(base):                       => partial(power,exponent=3)
#     return power(base, 3)

# function returns function

In [ ]:
acc_cbf = partial(AvgStatsCallback,accuracy)

# class AvgStatsCallback(Callback):
#     def __init__(self, metrics = accuracy):
#         self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)

In [ ]:
run = Runner(cb_funcs=acc_cbf)

In [ ]:
run.fit(1, learn)

train: [0.0632914892578125, tensor(0.9797)]
valid: [0.2720301025390625, tensor(0.9248)]


In [ ]:
#using cb.name in runner
# AvgStatsCallback -> avg_stats by name function and set in the runner
# then stick to runner with that name.
run.avg_stats.valid_stats.avg_stats

[0.15419449462890625, tensor(0.9538)]

## Export

In [ ]:
!python notebook2script.py 04_callbacks.ipynb

Converted 04_callbacks.ipynb to nb_04.py
